### Analysis V2
* AT&T and Verizon maps are done. 
* While being blocked with T-Mobile, I wanted to just check out how `utilities.load_unique_routes_df()` would work against the two finished maps. 
* Unfortunately I am blocked...Although testing  a few areas/routes with AT&T's map worked in the `multi_county_test.ipynb` file worked well, I ran into errors while working with certain routes
    * See section with HELP!

In [1]:
import dask.dataframe as dd
import dask_geopandas as dg
import numpy as np
import geopandas as gpd
import pandas as pd
import shapely.wkt
import utilities
from calitp import *
from shared_utils import geography_utils, utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### Load in Provider Maps
* These maps contain areas that *do not* have data coverage by providers. 
* Testing with AT&T since it seems to be the most reliable.

In [2]:
# verizon = gpd.read_parquet( "gs://calitp-analytics-data/data-analyses/cellular_coverage/verizon_all_counties.parquet")

In [3]:
att = gpd.read_parquet("gs://calitp-analytics-data/data-analyses/cellular_coverage/att_all_counties.parquet")

In [4]:
# tmobile = gpd.read_parquet("gs://calitp-analytics-data/data-analyses/cellular_coverage/tmobile_no_coverage_complete_CA.parquet")

In [5]:
# verizon.geometry.is_valid.all(), att.geometry.is_valid.all(),tmobile.geometry.is_valid.all(), 

In [6]:
# Make a copy of ATT for testing
# att2 = att.copy()

In [7]:
# Simplify the geometry
# att2["geometry"] = att2.geometry.simplify(tolerance=25)

In [8]:
# att2.shape

### Load in Routes 

In [9]:
unique_routes = utilities.load_unique_routes_df()

In [10]:
type(unique_routes)

geopandas.geodataframe.GeoDataFrame

In [11]:
len(unique_routes)

2829

In [ ]:
# Ensure CRS are the same
unique_routes = unique_routes.to_crs(att.crs)

In [ ]:
# B/c route names and route ids can be the same across different agencies,
# Add these 3 different columns so the route will have a unique identifier.
# Just in case I want to loop through everything
# unique_routes['long_route_name'] = unique_routes['route_name'] + ' ' + unique_routes['route_id'] + ' '  + ' ' + unique_routes['agency']

In [ ]:
unique_routes.long_route_name.nunique() == len(unique_routes) 

In [ ]:
len(unique_routes)

#### Testing with certain routes - Help! 
* Chose routes that gave me issues the first time around
* Getting an error when using the `att2` gdf which simplifies geometry.
    * `TopologyException: Input geom 1 is invalid: Hole lies outside shell at -124.1036 40.437482000000003`
    *  Works fine with original att df.

In [ ]:
# Routes from analysis v1 that didn't make sense
routes_wanted = [
    "Edmund D. Edelman Children’s Court Shuttle",
    "Oakland - San Francisco",
    "Foster City - Hillsdale Mall",
    "24 hour service daily",
    "Mammoth Lakes HWY 120E/395",
    "Avocado Heights/Bassett/West Valinda Shuttle",
    "Long Beach/San Pedro"
]

In [ ]:
unique_routes_subset = unique_routes[unique_routes["route_name"].isin(routes_wanted)]

In [ ]:
# Testing with original AT&T dataframe without simplifying geometry
# intersect_subset1 = gpd.overlay(unique_routes_subset, att, how="intersection")

In [ ]:
# intersect_subset1

In [ ]:
# Doesn't work with geometry simplify?
# intersect_subset2 = gpd.overlay(unique_routes_subset, att2, how="intersection")

In [ ]:
unique_routes_sample = unique_routes.sample(10).reset_index()

In [ ]:
unique_routes_sample.shape

In [ ]:
# Grab unique long route names for entire routes_df into a list
unique_routes_names_list = unique_routes_sample.long_route_name.unique().tolist()

In [ ]:
unique_routes_sample.plot()

In [ ]:
# Empty dataframe to hold each district after clipping
all_intersected_routes = pd.DataFrame()

In [ ]:
# Test with subsetted unique routes
for i in unique_routes_names_list:
    unique_routes_filtered  = unique_routes_sample[unique_routes_sample.long_route_name == i].reset_index(
            drop=True
    )

    # https://dask-geopandas.readthedocs.io/en/stable/docs/reference/api/dask_geopandas.GeoDataFrame.sjoin.html
    intersected = gpd.overlay(unique_routes_filtered, att, how="intersection")
    all_intersected_routes = pd.concat([all_intersected_routes, intersected], axis=0)
    print(f"Done intersecting & concatting for {i}")

In [ ]:
all_intersected_routes

In [ ]:
# Empty dataframe to hold each district after clipping
all_intersected_routes2 = pd.DataFrame()

#### Test a big chunk of routes dataframe - HELP!
* I get an error for certain routes even when intersecting with the original AT&T map
* This didn't happen in my `analysis_v1` so not sure why there's issues now?


In [16]:
def overlay_provider_unique_routes(provider: gpd.GeoDataFrame, 
                                  provider_name:str):
    
    # Empty dataframe to hold results
    all_intersected_routes = pd.DataFrame()

    for i in unique_routes_list:
        unique_routes_filtered  = unique_routes[unique_routes.long_route_name == i].reset_index(
                drop=True
        )
        try: 
            intersect = gpd.overlay(unique_routes_filtered, att, how="intersection")
            all_intersected_routes  = pd.concat([all_intersected_routes, intersect], axis=0)
        except: 
            pass 
            print(f"{i} didn't work")
    
    utils.geoparquet_gcs_export(all_intersected_routes, utilities.GCS_FILE_PATH, f"{provider_name}_intersected_routes")


In [ ]:
# Routes to drop that cause errors
routes_to_del = ['Sacramento 569  Amador Regional Transit System','Upcountry 570  Amador Regional Transit System',]